# `Sign Language Predection using CNN`🪧🤘

`Our approach:` 🧠🤯
1. Collect the images 
2. Import Libraries
3. Attribute Initialization
4. Use gpu
5. Image preprocessing
   1. resizing image to (224,224)
   1. splitting
6. input to CNN model 
7. change model to .h5
8. then to .tflite

In [2]:
#%pip install -U scikit-learn

In [7]:
#%pip install tensorflow --user

  Using cached tensorflow-2.13.0-cp38-cp38-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.13.0-cp38-cp38-win_amd64.whl (276.5 MB)
  Using cached protobuf-4.23.4-cp38-cp38-win_amd64.whl (422 kB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.17.0 requires protobuf<4,>=3.12, but you have protobuf 4.23.4 which is incompatible.
mysql-connector-python 8.0.31 requires protobuf<=3.20.1,>=3.11.0, but you have protobuf 4.23.4 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importing Libraries 📚

In [2]:
#imports
import os
import cv2
#import imghdr
#import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from matplotlib import pyplot as plt
from keras.applications import ResNet50
from sklearn.preprocessing import normalize
from tensorflow.keras.preprocessing import image
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
#----------------
import keras
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Dense,Flatten, Dropout
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.optimizers import SGD

## Attribute Initializing 📎

In [3]:
# Attributes Initialization
labels_dir = 'Dataset\Annotation'
classes_dir= 'Dataset\Signs'
image_exts = ['jpeg','jpg', 'bmp', 'png','JPG']                               #putting all the wanted image extensions
class_names = ['drink', 'eat', 'hello','help','love','morning','okay', 'please','stop','thankyou']
image_size = (224, 224)
batch_size = 32

## Use the GPU 💻

In [4]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
tf.config.list_physical_devices('GPU')

[]

In [ ]:
# import glob
# import shutil
# import os
# import xml.etree.ElementTree as ET

In [ ]:
# #for each class name loop on annotation find its equivalent label then find this file in images and add it to its file name 
# for x in class_names:
#   directory = labels_dir
#   print("class:",x)
#   # iterate over files in
#   # that directory
#   for filename in os.listdir(directory):
#       image_jpg=os.path.splitext(filename)[0]
#       print(image_jpg)
#       f = os.path.join(directory, filename)
#       # checking if it is a file
#       if os.path.isfile(f):
#           print("xml file :",f)
#           tree = ET.parse(f)
#           root = tree.getroot()
#           tag_name=root[6][0].text
#           print("tag:",tag_name)
#           file_name=image_jpg + ".jpg"
#           #extract the label
#           if (x==tag_name):
#             #get the file from images move to signs\class
#             src_dir=os.path.join(data_dir, file_name)
#             dst_dir = os.path.join(classes_dir, x)
#             print("al image gaya menan:",src_dir)
#             print("al image ray7a fan:",dst_dir)
#             shutil.copy(src_dir,dst_dir)




# Image Preprocessing🖼️

In [6]:
imageSize = (224, 224)
batchSize = 32

In [ ]:
# # we define an ImageDataGenerator object and
# # * set rescale to 1./255 to rescale the pixel values to the range [0, 1].
# data_generator = ImageDataGenerator(rescale=1./255)

# # we use the flow_from_directory method of the
# # ImageDataGenerator object to load a subset of the dataset
# # * we made resizing and image interpolation using bilinear
# gen_data = data_generator.flow_from_directory(
#     directory=classes_dir,
#     target_size=(224, 224),
#     batch_size=32, 
#     class_mode=None,
#     shuffle=False,
#     interpolation='bilinear'
# )

## Split Train and Test 🪓

In [7]:
gen_data_train =tf.keras.utils.image_dataset_from_directory(
    classes_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='rgb',
    batch_size=batchSize,
    image_size=imageSize,
    shuffle=True,
    seed=3,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,

)


Found 942 files belonging to 10 classes.
Using 754 files for training.


In [8]:
gen_data_test =tf.keras.utils.image_dataset_from_directory(
    classes_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=class_names,
    color_mode='rgb',
    batch_size=batchSize,
    image_size=imageSize,
    shuffle=True,
    seed=3,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,

)

Found 942 files belonging to 10 classes.
Using 188 files for validation.


In [ ]:
#print(gen_data_test[0])
#print(gen_data_test[1])

In [ ]:
# #visualization
# fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(10,10))
# for i in range(4):

#   # convert to unsigned integers for plotting
#   image = next(gen_data)[6]

#   # changing size from (1, 200, 200, 3) to (200, 200, 3) for plotting the image
#   image = np.squeeze(image)

#   # plot raw pixel data
#   ax[i].imshow(image)
#   ax[i].axis('off')

## CNN model 📦

In [9]:
#* CNN model
classifier = Sequential()
classifier.add(Conv2D(filters=32, kernel_size=(4,4),strides=(1,1),padding='same',input_shape=(224, 224, 3),activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(filters=64, kernel_size=(4,4),strides=(1,1),padding='same',activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(10, activation='softmax'))
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.fit(gen_data_train, epochs=20, batch_size=32)

Epoch 1/20
 4/24 [====>.........................] - ETA: 33s - loss: 7266.7588 - accuracy: 0.1094

KeyboardInterrupt: 

In [11]:
accuracy = classifier.evaluate(gen_data_test,batch_size=32)
print("Accuracy: ",accuracy[1])

6/6 [==============================] - 1s 165ms/step - loss: 122.6270 - accuracy: 0.1064
Accuracy:  0.10638298094272614


In [12]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      1568      
                                                                 
 dropout (Dropout)           (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      32832     
                                                                 
 dropout_1 (Dropout)         (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                   

## Save model to .h5 📱

In [13]:
classifier.save("trained_model.h5")

c:\Users\lenovo\anaconda3\envs\Elpo\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Save Model to .tflite 📲

In [14]:
# Load the trained Keras model from the file
loaded_model = keras.models.load_model("trained_model.h5")
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open("trained_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\lenovo\AppData\Local\Temp\tmppv6xvqnt\assets


INFO:tensorflow:Assets written to: C:\Users\lenovo\AppData\Local\Temp\tmppv6xvqnt\assets
